In [ ]:

!gdown --id 1oWqGyirJe85zUnrwX3BUxKUTQpxzXCEJ -O Training e test set Umberto.zip

import os
os.mkdir('/content/Training e test set Umberto')
!unzip /content/Training e test set Umberto.zip -d /content/Training e test set Umberto/


In [ ]:
import os
import pathlib
import pandas as pd
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="";

!pip install ktrain
import ktrain
from ktrain import text
MODEL_NAME="Musixmatch/umberto-commoncrawl-cased-v1"

In [ ]:
train_path= "../input/dati-di-training-e-di-test/haspeede2_dev_taskAB.csv"
test_path="../input/dati-di-training-e-di-test/haspeede2_reference_taskAB-tweets.csv"
tr_path=pathlib.Path(train_path)
te_path=pathlib.Path(test_path)
if tr_path.exists():
    print("Train data path set.")
else:
    raise SystemExit("Training Data Path does not exist.")

if te_path.exists():
    print("Test data path set.")
else:
    raise SystemExit("Test Data Path does not exist.")

In [ ]:
train_df= pd.read_csv(train_path, encoding='utf-8', sep="\t", header=None)
train_df=train_df.drop([0])
train_df.head()

test_df= pd.read_csv(test_path, encoding='utf-8', sep="\t",header=None)
test_df.head()
from datasets import Dataset

train=Dataset.from_pandas(train_df)
test=Dataset.from_pandas(test_df)


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
inputs_tr=tokenizer(train['1'],padding="max_length",truncation=True)
inputs_te=tokenizer(test['1'],padding="max_length",truncation=True)

def tokenize_function(examples):
    return tokenizer(examples["1"], padding="max_length", truncation=True)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test= test.map(tokenize_function, batched=True)



In [ ]:

tokenized_train = tokenized_train.remove_columns(["0"])
tokenized_train = tokenized_train.remove_columns(["1"])
tokenized_train = tokenized_train.remove_columns(["3"])
print(tokenized_train)
tokenized_train = tokenized_train.rename_column("2", "labels")
tokenized_train.set_format("torch")

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_train,batch_size=32)
eval_dataloader = DataLoader(tokenized_test, batch_size=32)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-4)

In [ ]:
from transformers import get_scheduler
from datasets import load_metric

num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

metric= load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

